# Circadian Dysregulation & Behavioral Symptom Burden

**PeriNeuroImmuneMap Analysis Pipeline**

---

**Correspondence:** Wei Qin (<wqin@sjtu.edu.cn>)


In [ ]:
from pathlib import Path

BASE_DIR = Path('.')
DATA_DIR = BASE_DIR / 'data'
OUTPUT_DIR = BASE_DIR / 'outputs'
FIGURE_DIR = OUTPUT_DIR / 'figures'
TABLE_DIR  = OUTPUT_DIR / 'tables'

for d in [DATA_DIR, OUTPUT_DIR, FIGURE_DIR, TABLE_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print('✓ Paths configured')


In [ ]:
import os, sys, base64, json, subprocess, tempfile, shutil
from pathlib import Path
from getpass import getpass
import nbformat

# -------------------------
# CONFIG (edit if needed)
# -------------------------
REPO_OWNER = "Sjtu-Fuxilab"
REPO_NAME  = "PeriNeuroImmuneMap"

# ✅ Your correct notebook path:
NOTEBOOK_LOCAL_PATH = r"D:\个人文件夹\Sanwal\Neuro\nc.ipynb"

# Where to place it inside the repo:
DEST_PATH_IN_REPO = "notebooks/nc.ipynb"   # change to "nc.ipynb" if you want it at repo root

# API limit safety (GitHub contents API is ~1MB; keep margin)
MAX_API_BYTES = 900_000

# -------------------------
# Helpers
# -------------------------
def ensure_requests():
    try:
        import requests
        return requests
    except Exception:
        print("Installing requests...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "requests"])
        import requests
        return requests

def gh_headers(token):
    return {
        "Authorization": f"token {token}",
        "Accept": "application/vnd.github+json",
    }

def gh_repo_info(requests, token):
    url = f"https://api.github.com/repos/{REPO_OWNER}/{REPO_NAME}"
    r = requests.get(url, headers=gh_headers(token))
    if r.status_code != 200:
        raise RuntimeError(f"Repo not accessible. HTTP {r.status_code}: {r.text}")
    return r.json()

def gh_get_sha_if_exists(requests, token, repo_path):
    url = f"https://api.github.com/repos/{REPO_OWNER}/{REPO_NAME}/contents/{repo_path}"
    r = requests.get(url, headers=gh_headers(token))
    if r.status_code == 200:
        return r.json().get("sha")
    if r.status_code == 404:
        return None
    raise RuntimeError(f"Failed checking existing file. HTTP {r.status_code}: {r.text}")

def upload_via_contents_api(requests, token, local_path, repo_path, commit_message):
    local_path = Path(local_path)
    data_bytes = local_path.read_bytes()
    content_b64 = base64.b64encode(data_bytes).decode("utf-8")

    sha = gh_get_sha_if_exists(requests, token, repo_path)

    url = f"https://api.github.com/repos/{REPO_OWNER}/{REPO_NAME}/contents/{repo_path}"
    payload = {"message": commit_message, "content": content_b64}
    if sha:
        payload["sha"] = sha

    r = requests.put(url, headers=gh_headers(token), json=payload)
    if r.status_code in (200, 201):
        return True, r.json()
    return False, r.text

def run(cmd, cwd=None):
    p = subprocess.run(cmd, cwd=cwd, text=True, capture_output=True)
    if p.returncode != 0:
        raise RuntimeError(f"Command failed:\n  {' '.join(cmd)}\n\nSTDOUT:\n{p.stdout}\n\nSTDERR:\n{p.stderr}")
    return p.stdout.strip()

def upload_via_git(token, default_branch, local_path, repo_path):
    # Requires git installed
    run(["git", "--version"])

    tmpdir = Path(tempfile.mkdtemp(prefix="gh_upload_"))
    try:
        # Clone using token (stored temporarily); then sanitize remote url after push
        clone_url_with_token = f"https://{token}@github.com/{REPO_OWNER}/{REPO_NAME}.git"
        print(f"Cloning into temp dir: {tmpdir}")
        run(["git", "clone", "--depth", "1", "--branch", default_branch, clone_url_with_token, str(tmpdir)])

        # Copy notebook into repo path
        target = tmpdir / repo_path
        target.parent.mkdir(parents=True, exist_ok=True)
        shutil.copy2(str(local_path), str(target))

        # Commit & push
        run(["git", "add", repo_path], cwd=tmpdir)
        # If no changes, git commit will fail; detect with status porcelain
        status = run(["git", "status", "--porcelain"], cwd=tmpdir)
        if not status.strip():
            print("No changes detected (remote already has identical file). Nothing to push.")
        else:
            # Set a local identity (won't affect your global git)
            run(["git", "config", "user.email", "actions@users.noreply.github.com"], cwd=tmpdir)
            run(["git", "config", "user.name", "Notebook Uploader"], cwd=tmpdir)
            run(["git", "commit", "-m", f"Update {repo_path}"], cwd=tmpdir)
            run(["git", "push", "origin", default_branch], cwd=tmpdir)
            print("Pushed changes via git.")

        # Sanitize remote URL to remove token from .git/config in temp clone
        run(["git", "remote", "set-url", "origin", f"https://github.com/{REPO_OWNER}/{REPO_NAME}.git"], cwd=tmpdir)
        return True
    finally:
        shutil.rmtree(tmpdir, ignore_errors=True)

# -------------------------
# Main
# -------------------------
print("=" * 80)
print("Upload complete notebook to GitHub (without outputs)")
print(f"Local notebook: {NOTEBOOK_LOCAL_PATH}")
print(f"Repo target:   https://github.com/{REPO_OWNER}/{REPO_NAME}  ->  {DEST_PATH_IN_REPO}")
print("=" * 80)

# Remove outputs from the notebook
def clear_notebook_outputs(nb_path):
    """Remove all outputs from the notebook."""
    with open(nb_path, 'r', encoding="utf-8") as f:
        nb = nbformat.read(f, as_version=4)
    
    for cell in nb.cells:
        cell['outputs'] = []
        cell['execution_count'] = None
    
    with open(nb_path, 'w', encoding="utf-8") as f:
        nbformat.write(nb, f)
    print("✅ Outputs removed from notebook.")

clear_notebook_outputs(NOTEBOOK_LOCAL_PATH)

nb_path = Path(NOTEBOOK_LOCAL_PATH)
if not nb_path.exists():
    raise FileNotFoundError(f"Notebook not found:\n  {NOTEBOOK_LOCAL_PATH}")

token = getpass("GitHub Personal Access Token (PAT): ").strip()
if not token:
    raise SystemExit("No token provided. Exiting.")

requests = ensure_requests()
repo = gh_repo_info(requests, token)
default_branch = repo.get("default_branch", "main")

size_bytes = nb_path.stat().st_size
print(f"\nNotebook size: {size_bytes:,} bytes")

# Try Contents API first if size is safe
if size_bytes <= MAX_API_BYTES:
    print("\nTrying GitHub Contents API upload...")
    ok, info = upload_via_contents_api(
        requests,
        token,
        nb_path,
        DEST_PATH_IN_REPO,
        commit_message=f"Upload notebook {DEST_PATH_IN_REPO}"
    )
    if ok:
        print("✅ Uploaded via Contents API.")
        print(f"Done: https://github.com/{REPO_OWNER}/{REPO_NAME}/blob/{default_branch}/{DEST_PATH_IN_REPO}")
    else:
        print("⚠️ Contents API upload failed; falling back to git method...")
        print(info if isinstance(info, str) else json.dumps(info, indent=2))
        print("\nTrying git clone/commit/push...")
        ok2 = upload_via_git(token, default_branch, nb_path, DEST_PATH_IN_REPO)
        if ok2:
            print("✅ Uploaded via git.")
            print(f"Done: https://github.com/{REPO_OWNER}/{REPO_NAME}/blob/{default_branch}/{DEST_PATH_IN_REPO}")
else:
    # Too large: go straight to git method
    print("\nNotebook is large; using git clone/commit/push (avoids Contents API size limits)...")
    ok2 = upload_via_git(token, default_branch, nb_path, DEST_PATH_IN_REPO)
    if ok2:
        print("✅ Uploaded via git.")
        print(f"Done: https://github.com/{REPO_OWNER}/{REPO_NAME}/blob/{default_branch}/{DEST_PATH_IN_REPO}")

print("\nAll done.")
